# Soils-revealed basemaps and layers

In [1]:
import ipyleaflet as ipyl
import geopandas as gpd
import ee
ee.Initialize()

**Political boundaries**

In [ ]:
countries = gpd.read_file('../data/gadm36_0/gadm36_0.shp')
countries['simplify_geom'] = countries['geometry'].apply(lambda x: x.simplify(0.002))
Argentina = countries[countries['NAME_0'] == 'Argentina']

locations = []
for i in range(len(countries)):
    geometry = countries['simplify_geom'].iloc[i]
    if geometry.type == 'Polygon':
        x, y = geometry.exterior.coords.xy
        locations.append(list(zip(list(y), list(x))))
    if geometry.type == 'MultiPolygon':
        for polygon in geometry:  
            x, y = polygon.exterior.coords.xy
            locations.append(list(zip(list(y), list(x))))
    
locationA = []
geometry = Argentina['simplify_geom'].iloc[0]
for polygon in geometry:  
    x, y = polygon.exterior.coords.xy
    locationA.append(list(zip(list(y), list(x))))

## Basemaps

In [2]:
topography_url = 'https://api.mapbox.com/styles/v1/casius/ck801p48x1hsd1iqwe67w1lac/tiles/256/{z}/{x}/{y}@2x?access_token=pk.eyJ1IjoiY2FzaXVzIiwiYSI6ImJDMkpucTQifQ.5rm4_TsT8_PH8TzOY2V3FQ'
topography_basemap = ipyl.TileLayer(
    url=topography_url,
    layers='topography',
    format='image/png',
    name='Topography',
    opacity=1
)

landsat_url = 'https://storage.cloud.google.com/skydipper-landsat-cache/2018/{z}/{x}/{y}.png'
landsat_basemap = ipyl.TileLayer(
    url=landsat_url,
    layers='landsat',
    format='image/png',
    name='Landsat',
    opacity=0.5
)

In [3]:
center = (0, 0)
m = ipyl.Map(center=(center), zoom=2)


m.add_layer(topography_basemap)
m.add_layer(landsat_basemap)

control = ipyl.LayersControl(position='topright')
m.add_control(control)
m.add_control(ipyl.FullScreenControl())
m

Map(center=[0, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text'…

## Soil Layers

In [4]:
def get_tiles(images, sld_intervals, names, locations=None, opacity=1, basemap='topography', center = (0, 0), zoom=2):
    ee_tiles = 'https://earthengine.googleapis.com/map/{mapid}/{{z}}/{{x}}/{{y}}?token={token}' 
    
    m = ipyl.Map(center=(center), zoom=zoom)
    
    if basemap == 'topography':
        m.add_layer(topography_basemap)
    if basemap == 'landsat':
        m.add_layer(landsat_basemap)
        
    for n, image in enumerate(images):
        image = image.sldStyle(sld_intervals[n])
        mapid = image.getMapId()
        tiles_url = ee_tiles.format(**mapid)
        
        tile_layer = ipyl.TileLayer(
            url=tiles_url,
            format='image/png',
            name=names[n],
            opacity=opacity
        )
    
        m.add_layer(tile_layer)
        
    if locations:
        for n, location in enumerate(locations):
            if n == 0:
                polygon = ipyl.Polygon(locations=location, color="black", weight=1, fill=False, name='Countries')
                m.add_layer(polygon)
            else:
                polygon = ipyl.Polygon(locations=location, color="black", weight=3, fill=False, name='Argentina')
                m.add_layer(polygon)
    
        
    
    control = ipyl.LayersControl(position='topright')
    m.add_control(control)
    m.add_control(ipyl.FullScreenControl())
    
    return m

### Historic SOC stocks

In [7]:
images = []
sld_intervals = []
names = []

**Current SOC stocks (2010)**

In [6]:
name='Current SOC stocks (0-200 cm)'

image = ee.Image('users/iker/SOC_maps/SOCS_0_200cm_year_2010AD_10km')

sld_interval = '<RasterSymbolizer>' + \
'<ColorMap extended="false" type="ramp">' + \
'<ColorMapEntry color="#E18D67" quantity="10"  opacity="1" />' + \
'<ColorMapEntry color="#CB5A3A" quantity="40"  />' + \
'<ColorMapEntry color="#9D4028" quantity="80" />' + \
'<ColorMapEntry color="#6D2410" quantity="160"  />' + \
'<ColorMapEntry color="#380E03" quantity="400"  />' + \
'</ColorMap>' + '</RasterSymbolizer>'

images.append(image)
sld_intervals.append(sld_interval)
names.append(name)

**Historic SOC stocks**

In [ ]:
name='Historic SOC stocks (0-200 cm)'

image = ee.Image('users/iker/SOC_maps/SOCS_0_200cm_year_NoLU_10km')

sld_interval = '<RasterSymbolizer>' + \
'<ColorMap extended="false" type="ramp">' + \
'<ColorMapEntry color="#E18D67" quantity="10"  opacity="1" />' + \
'<ColorMapEntry color="#CB5A3A" quantity="40"  />' + \
'<ColorMapEntry color="#9D4028" quantity="80" />' + \
'<ColorMapEntry color="#6D2410" quantity="160"  />' + \
'<ColorMapEntry color="#380E03" quantity="400"  />' + \
'</ColorMap>' + '</RasterSymbolizer>'

images.append(image)
sld_intervals.append(sld_interval)
names.append(name)

**Historic SOC stocks change**

In [ ]:
name='Historic SOC stocks change (0-200 cm)'
imageNoLU = ee.Image('users/iker/SOC_maps/SOCS_0_200cm_year_NoLU_10km')
image2010 = ee.Image('users/iker/SOC_maps/SOCS_0_200cm_year_2010AD_10km')

image = image2010.select('b1').subtract(imageNoLU.select('b1'))

sld_interval = '<RasterSymbolizer>' + \
'<ColorMap extended="false" type="ramp">' + \
'<ColorMapEntry color="#B30200" quantity="-80"  opacity="1" />' + \
'<ColorMapEntry color="#E34A33" quantity="-40"  />' + \
'<ColorMapEntry color="#FC8D59" quantity="-20" />' + \
'<ColorMapEntry color="#FDCC8A" quantity="-10"  />' + \
'<ColorMapEntry color="#FFFFCC" quantity="0"  />' + \
'<ColorMapEntry color="#A1DAB4" quantity="10" />' + \
'<ColorMapEntry color="#31B3BD" quantity="20"  />' + \
'<ColorMapEntry color="#1C9099" quantity="40" />' + \
'<ColorMapEntry color="#066C59" quantity="80"  />' + \
'</ColorMap>' + '</RasterSymbolizer>'

images.append(image)
sld_intervals.append(sld_interval)
names.append(name)

In [ ]:
get_tiles(images, sld_intervals, names)

### Recent SOC stocks

In [16]:
images = []
sld_intervals = []
names = []

**Global SOC stock (0-5 cm)** ([gee](https://code.earthengine.google.com/e1b2826e83fd5463b4662872a7649c6d?asset=users%2Fiker%2FSOC_maps%2FSOC_concentration_q50))

In [17]:
name='Global SOC stock (0-5 cm)'

image = ee.Image('users/resourcewatch/foo_054_soil_organic_carbon_stocks')

sld_interval = '<RasterSymbolizer>' + \
'<ColorMap extended="false" type="ramp">' + \
'<ColorMapEntry color="#E18D67" quantity="5"  opacity="1" />' + \
'<ColorMapEntry color="#CB5A3A" quantity="20"  />' + \
'<ColorMapEntry color="#9D4028" quantity="40" />' + \
'<ColorMapEntry color="#6D2410" quantity="60"  />' + \
'<ColorMapEntry color="#380E03" quantity="80"  />' + \
'</ColorMap>' + '</RasterSymbolizer>'

images.append(image)
sld_intervals.append(sld_interval)
names.append(name)

**Argentina SOC stock (0-30 cm)** ([gee](https://code.earthengine.google.com/3cd64e858784a1a04f74a7c615162c0b?asset=users%2Fiker%2FSOC_maps%2FSOC_concentration_q50))

In [18]:
name='Argentina SOC stock (0-30 cm)'

image = ee.Image(ee.ImageCollection('users/iker/SOC_maps/SOC_stock_0_30').filterDate('2017-01-01', '2017-12-31').first())

sld_interval = '<RasterSymbolizer>' + \
'<ColorMap extended="false" type="ramp">' + \
'<ColorMapEntry color="#E18D67" quantity="5"  opacity="1" />' + \
'<ColorMapEntry color="#CB5A3A" quantity="20"  />' + \
'<ColorMapEntry color="#9D4028" quantity="50" />' + \
'<ColorMapEntry color="#6D2410" quantity="75"  />' + \
'<ColorMapEntry color="#380E03" quantity="200"  />' + \
'</ColorMap>' + '</RasterSymbolizer>'

images.append(image)
sld_intervals.append(sld_interval)
names.append(name)

**Argentina SOC stock change (0-30 cm)** ([gee](https://code.earthengine.google.com/b927128c41e5c298145000ad67104500?asset=users%2Fiker%2FSOC_maps%2FSOC_concentration_q50))

In [19]:
name='Argentina SOC stock change (0-30 cm)'

image1982 = ee.Image(ee.ImageCollection('users/iker/SOC_maps/SOC_stock_0_30').filterDate('1982-01-01', '1982-12-31').first())
image2017 = ee.Image(ee.ImageCollection('users/iker/SOC_maps/SOC_stock_0_30').filterDate('2017-01-01', '2017-12-31').first())

image = image2017.select('b1').subtract(image1982.select('b1'))

sld_interval = '<RasterSymbolizer>' + \
'<ColorMap extended="false" type="ramp">' + \
'<ColorMapEntry color="#B30200" quantity="-30"  opacity="1" />' + \
'<ColorMapEntry color="#E34A33" quantity="-20"  />' + \
'<ColorMapEntry color="#FC8D59" quantity="-10" />' + \
'<ColorMapEntry color="#FDCC8A" quantity="-5"  />' + \
'<ColorMapEntry color="#FFFFCC" quantity="0"  />' + \
'<ColorMapEntry color="#A1DAB4" quantity="5" />' + \
'<ColorMapEntry color="#31B3BD" quantity="10"  />' + \
'<ColorMapEntry color="#1C9099" quantity="20" />' + \
'<ColorMapEntry color="#066C59" quantity="30"  />' + \
'</ColorMap>' + '</RasterSymbolizer>'

images.append(image)
sld_intervals.append(sld_interval)
names.append(name)

**Argentina SOC concentration (0-5 cm)**

In [ ]:
name='Argentina SOC concentration (0-5 cm)'

image = ee.Image(ee.ImageCollection('users/iker/SOC_maps/SOC_concentration_q50').filterDate('2017-01-01', '2017-12-31').first())
image = image.divide(10).select('b1')

sld_interval = '<RasterSymbolizer>' + \
'<ColorMap extended="false" type="ramp">' + \
'<ColorMapEntry color="#E18D67" quantity="5"  opacity="1" />' + \
'<ColorMapEntry color="#CB5A3A" quantity="10"  />' + \
'<ColorMapEntry color="#9D4028" quantity="20" />' + \
'<ColorMapEntry color="#6D2410" quantity="30"  />' + \
'<ColorMapEntry color="#380E03" quantity="40"  />' + \
'</ColorMap>' + '</RasterSymbolizer>'

images.append(image)
sld_intervals.append(sld_interval)
names.append(name)

**Argentina SOC concentration change (0-5 cm)**

In [ ]:
name='Argentina SOC concentration change (0-5 cm)'

image1982 = ee.Image(ee.ImageCollection('users/iker/SOC_maps/SOC_concentration_q50').filterDate('1982-01-01', '1982-12-31').first())
image2017 = ee.Image(ee.ImageCollection('users/iker/SOC_maps/SOC_concentration_q50').filterDate('2017-01-01', '2017-12-31').first())

image1982 = image1982.divide(10).select('b1')
image2017 = image2017.divide(10).select('b1')

image = image2017.select('b1').subtract(image1982.select('b1'))

sld_interval = '<RasterSymbolizer>' + \
'<ColorMap extended="false" type="ramp">' + \
'<ColorMapEntry color="#B30200" quantity="-3"  opacity="1" />' + \
'<ColorMapEntry color="#E34A33" quantity="-2"  />' + \
'<ColorMapEntry color="#FC8D59" quantity="-1" />' + \
'<ColorMapEntry color="#FCC8DA" quantity="-0.5"  />' + \
'<ColorMapEntry color="#FFFFCC" quantity="0"  />' + \
'<ColorMapEntry color="#A1DAB4" quantity="0.5" />' + \
'<ColorMapEntry color="#31B3BD" quantity="1"  />' + \
'<ColorMapEntry color="#1C9099" quantity="2" />' + \
'<ColorMapEntry color="#066C59" quantity="3"  />' + \
'</ColorMap>' + '</RasterSymbolizer>'

images.append(image)
sld_intervals.append(sld_interval)
names.append(name)

In [20]:
get_tiles(images, sld_intervals, names)

Map(center=[0, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text'…

### Future SOC stocks

## Contextual Layers
### Land Use

**Global Land Cover**

In [6]:
images = []
sld_intervals = []
names = []

In [7]:
name='Global Land Cover (2001)'

image = ee.Image('users/resourcewatchlandcover/esa_landcover_ipcc/ESA_2001_ipcc')

sld_interval = '<RasterSymbolizer>' + \
'<ColorMap type="values" extended="false">' + \
'<ColorMapEntry color="#000000" quantity="0" opacity="0" />'+ \
'<ColorMapEntry color="#DAC019" quantity="1" opacity="1" />'+ \
'<ColorMapEntry color="#09630c" quantity="2" />'+ \
'<ColorMapEntry color="#3fa02c" quantity="3"/>'+ \
'<ColorMapEntry color="#159578" quantity="4" />'+ \
'<ColorMapEntry color="#c11812" quantity="5" />'+ \
'<ColorMapEntry color="#956314" quantity="6"/>'+ \
'<ColorMapEntry color="#c2e575" quantity="7" />'+ \
'<ColorMapEntry color="#E7E48C" quantity="8" />'+ \
'<ColorMapEntry color="#07629D" quantity="210" />'+ \
'<ColorMapEntry color="#FFFFFF" quantity="220" />'+ \
'</ColorMap>' + '</RasterSymbolizer>'

images.append(image)
sld_intervals.append(sld_interval)
names.append(name)

get_tiles([image], [sld_interval], [name],opacity=1)

Map(center=[0, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text'…

**Tree Cover Loss**

In [ ]:
name='Tree Cover Loss (2010)'

image_url = 'https://production-api.globalforestwatch.org/v1/true-color-tiles/loss/{z}/{x}/{y}?startYear=2001&endYear=2010'

m = ipyl.Map(center=((0,0)), zoom=2)
    
m.add_layer(topography_basemap)
        
tile_layer = ipyl.TileLayer(
    url=image_url,
    format='image/png',
    name=name,
    opacity=1
)
    
m.add_layer(tile_layer)
    
control = ipyl.LayersControl(position='topright')
m.add_control(control)
m.add_control(ipyl.FullScreenControl())
    
m

**Cropland and Pasture Area**

In [ ]:
name='Cropland and Pasture Area'

image = ee.Image('users/resourcewatch/foo_049_cropland_and_pasture_area2000')

sld_interval = '<RasterSymbolizer> <ChannelSelection> <GrayChannel> <SourceChannelName>1</SourceChannelName> </GrayChannel> </ChannelSelection>' + \
'<ColorMap  type="ramp" extended="false">' + \
'<ColorMapEntry color="#d9f0a3" quantity="0.01" opacity="0" />' + \
'<ColorMapEntry color="#d9f0a3" quantity="0.2" />' + \
'<ColorMapEntry color="#addd8e" quantity="0.4" />' + \
'<ColorMapEntry color="#78c679" quantity="0.6" />' + \
'<ColorMapEntry color="#31a354" quantity="0.8" />' + \
'<ColorMapEntry color="#006837" quantity="1" />' + \
'</ColorMap>' + '</RasterSymbolizer>'

images.append(image)
sld_intervals.append(sld_interval)
names.append(name)

get_tiles([image], [sld_interval], [name])

**Wetlands and Waterbodies**

In [ ]:
name='Wetlands and Waterbodies'

image = ee.Image('users/resourcewatch/wat_039_wetlands_waterbodies')

sld_interval = '<RasterSymbolizer>' + \
'<ColorMap  type="values" extended="false">' + \
'<ColorMapEntry color="#6DB4D9" quantity="1" label="" opacity="1" />' + \
'<ColorMapEntry color="#1f78b4" quantity="2" label="" />' + \
'<ColorMapEntry color="#08306b" quantity="3" label="" />' + \
'<ColorMapEntry color="#80B160" quantity="4" label="" />' + \
'<ColorMapEntry color="#33a02c" quantity="5" label="" />' + \
'<ColorMapEntry color="#ED6F6E" quantity="6" label="" />' + \
'<ColorMapEntry color="#e31a1c" quantity="7" label="" />' + \
'<ColorMapEntry color="#D08522" quantity="8" label="" />' + \
'<ColorMapEntry color="#FFAE00" quantity="9" label="" />' + \
'<ColorMapEntry color="#D392F3" quantity="10" label="" />' + \
'<ColorMapEntry color="#6a3d9a" quantity="11" label="" />' + \
'<ColorMapEntry color="#b15928" quantity="12" label="" />' + \
'</ColorMap>' + '</RasterSymbolizer>'

images.append(image)
sld_intervals.append(sld_interval)
names.append(name)

get_tiles([image], [sld_interval], [name])

**Urban Built-Up Area**

In [ ]:
name='Urban Built-Up Area (2014)'

image = ee.Image('projects/resource-watch-gee/cit_033a_urban_built_up_area_30m_mosaic')

sld_interval = '<RasterSymbolizer> <ChannelSelection> <GrayChannel> <SourceChannelName>b1</SourceChannelName> </GrayChannel> </ChannelSelection>' + \
'<ColorMap  type="values" extended="false">' + \
'<ColorMapEntry color="#F03B20" quantity="6" label="" />' + \
'<ColorMapEntry color="#FEB24C" quantity="5" label="" />' + \
'<ColorMapEntry color="#FFEDA0" quantity="4" label="" />' + \
'<ColorMapEntry color="#f7f7f7" quantity="3" label="" />' + \
'</ColorMap></RasterSymbolizer>'

images.append(image)
sld_intervals.append(sld_interval)
names.append(name)

get_tiles([image], [sld_interval], [name])